# Community Engagement Scores

This notebook demonstrates how to use the package `ces` for calculating community engagement scores with data exported from the Swae portal of SingularityNET.

In [ ]:
import os

from ces import swae_analysis as swa

## Define input and output locations

In [ ]:
in_dir = os.path.join('..', 'tests', 'input')
in_zip_filepath = os.path.join(in_dir, 'raw.zip')
in_csv_filepath = os.path.join(in_dir, 'dfr2_results_extracted.csv')

In [ ]:
out_dir = 'output'
out_sqlite_filepath = os.path.join(out_dir, 'deep_funding_round_2.sqlite')
out_excel_filepath = os.path.join(out_dir, 'deep_funding_round_2.xlsx')

## Convert Swae data to SQLite

Extract, transform and load (ETL) data from multiple JSON files into a single SQLite database

In [ ]:
# in multiple steps
json_data = swa.extract_swae_data(in_zip_filepath)
tabular_data = swa.transform_swae_data(json_data)
con = swa.load_sqlite(tabular_data, out_sqlite_filepath)

In [ ]:
# in one step
con = swa.zip_to_sqlite(in_zip_filepath, out_sqlite_filepath, filters_on=False)

Store the SQLite database optionally also in an Excel file with multiple sheets

In [ ]:
swa.sqlite_to_excel(con, out_excel_filepath)

## Filter by missions

Which proposals, comments, ratings and reactions belong to the chosen missons? Which users were active in them?

In [ ]:
all_missions = swa.get_missions(con)
selected_missions = [m for m in all_missions if m[1].startswith('Round 2')]
selected_mission_ids = [m[0] for m in selected_missions]
selected_mission_ids

In [ ]:
filter_id = 1
swa.create_filter_views(con, filter_id, selected_mission_ids, extra_time_in_days=filter_id)

## Count user activities

What proposals, comments, ratings and reactions did each user create and receive?

In [ ]:
swa.create_counts_table(con, filter_id)

## Calculate contribution scores

How are the user activity counts converted into contribution scores? What values were given to the available variables?

In [ ]:
variables_dfr2 = {
    'proposals_created': 0,

    'ratings_created': 0,
    'ratings_received': 0,

    'comments_created': 3,
    'comments_received': 0,

    'upvote_reactions_created': 0,
    'downvote_reactions_created': 0,
    'celebrate_reactions_created': 0,
    'clap_reactions_created': 0,
    'curious_reactions_created': 0,
    'genius_reactions_created': 0,
    'happy_reactions_created': 0,
    'hot_reactions_created': 0,
    'laugh_reactions_created': 0,
    'love_reactions_created': 0,

    'anger_reactions_created': 0,
    'sad_reactions_created': 0,

    'upvote_reactions_received': 2,
    'downvote_reactions_received': -3,
    
    'celebrate_reactions_received': 2,
    'clap_reactions_received': 2,
    'curious_reactions_received': 2,
    'genius_reactions_received': 2,
    'happy_reactions_received': 2,
    'hot_reactions_received': 2,
    'laugh_reactions_received': 2,
    'love_reactions_received': 2,
    
    'anger_reactions_received': -2,
    'sad_reactions_received': -2,
    
    'fraction_of_engagement_scores_for_highly_rated_proposals': 0.0,
}

In [ ]:
variables_id = 1
swa.create_engagement_score_table(con, filter_id, variables_id, variables_dfr2)

In [ ]:
data = swa.get_engagement_scores(con, filter_id, variables_id)

import pandas as pd
dfr2_auto = pd.DataFrame(data, columns=['user_id', 'engagement_score'])

## Reward: Distribute rewards by user rank

### a) Original distribution by grouping users into tiers

In [ ]:
dfr2_man = pd.read_csv(in_csv_filepath)
dfr2_man = dfr2_man[dfr2_man['contribution_score'] > 0.0]
dfr2_man = dfr2_man.sort_values(by=['contribution_score', 'user_id'], ascending=[False, False])
dfr2_man.reset_index(drop=True, inplace=True)
dfr2_man.index = dfr2_man.index + 1

dfr2_man[['user_id', 'contribution_score']]

In [ ]:
(dfr2_auto.values == dfr2_man[['user_id', 'contribution_score']].values).all()

In [ ]:
from ces.swae_analysis.visualize import _create_bar_plot as plot

fig = plot(dfr2_man.index, dfr2_man['contribution_score'], 'User rank', 'Engagement score', 'black', inline=True)
fig = plot(dfr2_man.index, dfr2_man['contribution_score_rewards'], 'User rank', 'AGIX', 'blue', inline=True)
fig = plot(dfr2_man.index, dfr2_man['voting_weight'], 'User rank', 'Voting weight', 'green', inline=True)

### b) New distribution by applying a continuous function

In [ ]:
distribution_id = 1

filtered_user_ids = [
    'jan.horlings@singularitynet.io',
    'janhorlings@gmail.com',
]
swa.create_rewards_table(con, filter_id, variables_id, distribution_id,
                         threshold_percentile=10,
                         filtered_user_ids=filtered_user_ids,
                         #function_agix_reward='x**3',
                         #function_voting_weight='exp(x/10)'
)
figures = swa.plot_rewards(con, filter_id, variables_id, distribution_id, inline=True)

## Network analysis

The following is a preliminary attempt to calculate community engagement scores with network analysis and centrality measures.

In [ ]:
import gravis as gv
import networkx as nx

In [ ]:
mission_ids = ['c2c5e5c4-71c4-41cd-9fcf-e2fb3dbde8ab']

dg = swa.sqlite_to_graph(con, mission_ids)
gv.d3(dg, show_node_label=False)

Calculate centrality values and visualize them as node sizes

In [ ]:
node_to_centrality = nx.laplacian_centrality(dg)
for nd, val in node_to_centrality.items():
    dg.nodes[nd]['size'] = val * 1000

fig = gv.d3(dg, show_node_label=False, node_drag_fix=True, node_hover_neighborhood=True)
fig